In [1]:
%matplotlib inline
from __future__ import print_function

try:
    xrange
except NameError:
    xrange = range

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial as sp
import scipy.sparse as sparse
from sklearn.decomposition import NMF, TruncatedSVD #皮膚地、徳一

<h2>レコメンデーション</h2>

レコメンデーションの最も一般的なアルゴリズムである協調フィルタリングを紹介します。
まずは簡単なダミーデータで説明していきます。

In [2]:
rating_matrix = np.array([[2, 5, 1, 1, 0, 1, 2, 4],
                          [1, 5, 2, 1, 4, 0, 0, 3],
                          [0, 3, 3, 0, 1, 1, 1, 1],
                          [5, 2, 2, 3, 1, 0, 0, 4],
                          [5, 3, 3, 4, 1, 0, 0, 5],
                          [1, 4, 3, 2, 5, 1, 0, 1],
                          [0, 0, 0, 0, 0, 0, 0, 2],
                          [0, 4, 0, 0, 0, 0, 0, 0]])

In [58]:
len(rating_matrix)

8

1行目のユーザーと似ているユーザーを探しましょう。

In [59]:
user_similarity = []
for i in range(len(rating_matrix)):
    # cosine類似度　= 1 - cosine距離
    sim = 1 - sp.distance.cosine(rating_matrix[0], rating_matrix[i])
    user_similarity.append(sim)
    
user_similarity = np.array(user_similarity)
user_similarity

array([ 1.        ,  0.77831178,  0.73914049,  0.7402121 ,  0.78215389,
        0.58777469,  0.5547002 ,  0.69337525])

topNで指定した類似度の高いユーザーを探しましょう。

In [4]:
topN = 4

# userのインデックス（行番号）をNumpy Arrayで作っておきます
user_idx = np.array(range(0, len(rating_matrix)))

# 類似度の低い順にソートした結果のインデックスを用意して、降順に並び替えます
arg_sort = np.argsort(user_similarity)
arg_sort = arg_sort[::-1]

# 自分自身を除いた類似度の高い4人を選びます
selected_idx = arg_sort[1:topN+1]

selected_user_similarity = user_similarity[selected_idx]
selected_rating_matrix = rating_matrix[selected_idx]
selected_rating_matrix

array([[5, 3, 3, 4, 1, 0, 0, 5],
       [1, 5, 2, 1, 4, 0, 0, 3],
       [5, 2, 2, 3, 1, 0, 0, 4],
       [0, 3, 3, 0, 1, 1, 1, 1]])

In [60]:
arg_sort

array([0, 4, 1, 3, 2, 7, 5, 6], dtype=int64)

どの商品をオススメするか？ここでは平均類似度を使ってみましょう。

In [61]:
avg_score = []
for col_idx in range(selected_rating_matrix.shape[1]):
    weight_score = \
     sum(selected_rating_matrix[:, col_idx] * selected_user_similarity)
    similarity_sum = \
     sum(selected_user_similarity[selected_user_similarity > 0])
    avg_score.append(weight_score / similarity_sum)

ValueError: operands could not be broadcast together with shapes (4,) (20,) 

In [6]:
for i, v in enumerate(avg_score):
    print(str(i) + 'th item  ', v)

0th item   2.76008004028
1th item   3.26857245915
2th item   2.50045570209
3th item   2.01576643714
4th item   1.76811675706
5th item   0.243152856729
6th item   0.243152856729
7th item   3.27180535615


上記だと具体的な商品名がないので、わかりにくいですよね（でも雰囲気は伝わりましたか？）。もう少し具体的なデータでやってみましょう。

<h2>某ニュースアプリのテーマのフォロー状況を模したデータで協調フィルタリング</h2>

datasetフォルダにあるuser_topic_follow_dummy.csvを読み込みましょう。

In [7]:
data = pd.read_csv('dataset/user_topic_follow_dummy.csv', encoding='utf8')
data.drop_duplicates(keep='last', inplace=True)
print(data.shape)
data.head()

(339323, 2)


,user_id,topic_name
0,25126,(株)アップル
1,26285,(株)電通
2,15409,.NET_Framework
3,30466,.NET_Framework
4,30878,.NET_Framework


この後、Pandasのpivotを使って、User x Itemの行列を作りますので、ratingの列を新たに作成し、１.0を格納しておきます。

In [8]:
data['rating'] = 1.0

In [28]:
data

,user_id,topic_name,rating
0,25126,(株)アップル,1.0
1,26285,(株)電通,1.0
2,15409,.NET_Framework,1.0
3,30466,.NET_Framework,1.0
4,30878,.NET_Framework,1.0
5,36212,.NET_Framework,1.0
6,29520,0増5減,1.0
7,24,100円ショップ,1.0
8,1339,100円ショップ,1.0
9,2492,100円ショップ,1.0


実際に User x Itemの行列を作成します。

In [9]:
rating_matrix = \
 data.pivot(index='user_id', columns='topic_name', values='rating')
rating_matrix.fillna(0, inplace=True)

In [10]:
topic_list = np.array(rating_matrix.columns)
user_list = np.array(rating_matrix.index)

In [3]:
rating_matrix_ar = np.array(rating_matrix)
rating_matrix_ar

NameError: name 'np' is not defined

さて、user_id=1の方に対するオススメトピックを探してみましょう。

In [12]:
already_followed_topic = \
 np.array(data[data['user_id'] == user_list[0]]['topic_name'])
data[data['user_id'] == user_list[0]]

,user_id,topic_name,rating
40826,1,MacBook,1.0
187716,1,一人旅,1.0
211960,1,吉祥寺,1.0
287441,1,築地市場,1.0
327071,1,離乳食,1.0


実際にユーザーごとの類似度を計算しましょう。

In [1]:
def get_cosine_similarity(x, y):
    return 1- sp.distance.cosine(x, y)

def get_jaccard_similarity(x, y):
    return 1 - sp.distance.jaccard(x, y)

user_similarity = []
target_user_row = rating_matrix_ar[0]
for row in rating_matrix_ar:
    sim = get_jaccard_similarity(target_user_row, row)
    user_similarity.append(sim)
user_similarity = np.array(user_similarity)

NameError: name 'rating_matrix_ar' is not defined

In [2]:
rating_matrix_ar[:,0]

NameError: name 'rating_matrix_ar' is not defined

In [14]:
user_similarity

array([ 1.        ,  0.        ,  0.14285714, ...,  0.        ,
        0.08333333,  0.05263158])

類似度の高いユーザーTopNを抽出します。

In [15]:
topN = 20
idx = user_similarity.argsort()[::-1][1:topN+1]
selected_user_similarity = user_similarity[idx]
selected_rating = rating_matrix_ar[idx]

平均類似度を計算しましょう。

In [16]:
avg_score = []
for col_idx in range(selected_rating.shape[1]):
    weight_score = sum(selected_rating[:, col_idx] * selected_user_similarity)
    similarity_sum = \
     sum(selected_user_similarity[selected_user_similarity > 0])
    avg_score.append(weight_score / similarity_sum)
avg_score = np.array(avg_score)

そして、平均類似度の高い上位10テーマをオススメとして表示させます。

In [17]:
recommend_num = 5
counter = 0
for recommended_topic in topic_list[avg_score.argsort()[::-1]]:
    if recommended_topic not in already_followed_topic:
        print(recommended_topic)
        counter += 1
        if recommend_num <= counter:
            break

インテリジェント・デザイン
映画
音楽
デート
副業


どうでしょうか？これを気に入ってくれそうですか？

<h2>次元削減を行ってみましょう</h2>

ここでは、以下2つの手法を紹介します。
* SVD
* Non Negative Matrix Factorization

<h3>特異値分解(Singular Value Decomposition)による次元圧縮を使ったレコメンデーション</h3>

まず、SVDを初期化します。

In [18]:
svd = TruncatedSVD(n_components=20)

Scipy Sparse行列にしておきます。

In [19]:
rating_matrix_sparse = sparse.lil_matrix(rating_matrix_ar)

データを適用させます。

In [20]:
rating_matrix_svd = svd.fit_transform(rating_matrix_sparse)

あとは、前回と同じです。似ているユーザーを探して平均類似度の高いトピックをお勧めしましょう。

In [21]:
user_similarity = []
target_user_svd = rating_matrix_svd[0]
for row in rating_matrix_svd:
    sim = get_cosine_similarity(target_user_svd, row)
    user_similarity.append(sim)
user_similarity = np.array(user_similarity)

In [22]:
topN = 20
idx = user_similarity.argsort()[::-1][1:topN+1]
selected_user_similarity = user_similarity[idx]
selected_rating = rating_matrix_ar[idx]

avg_score = []
for col_idx in range(selected_rating.shape[1]):
    weight_score = sum(selected_rating[:, col_idx] * selected_user_similarity)
    similarity_sum = \
     sum(selected_user_similarity[selected_user_similarity > 0])
    avg_score.append(weight_score / similarity_sum)
avg_score = np.array(avg_score)

recommend_num = 5
counter = 0
for recommended_topic in topic_list[avg_score.argsort()[::-1]]:
    if recommended_topic not in already_followed_topic:
        print(recommended_topic)
        counter += 1
        if recommend_num <= counter:
            break

インテリジェント・デザイン
ロードバイク
Twitter
ゲーム
PlayStation_4


<h3>非負値行列分解(Non Negative Matrix Factorization)による次元圧縮を使ったレコメンデーション</h3>

In [38]:
nmf = NMF(n_components=10, random_state=1236, init='random')

In [39]:
rating_matrix_sparse

<29332x14167 sparse matrix of type '<class 'numpy.float64'>'
	with 339323 stored elements in LInked List format>

In [40]:
rating_matrix_nmf = nmf.fit_transform(rating_matrix_sparse)

In [41]:
rating_matrix_nmf.shape

(29332, 10)

In [42]:
user_similarity = []
target_user_nmf = rating_matrix_nmf[0]
for row in rating_matrix_nmf:
    if sum(row) == 0:
        sim = 0
        user_similarity.append(sim)
    else:
        sim = get_cosine_similarity(target_user_nmf, row)
        user_similarity.append(sim)
user_similarity = np.array(user_similarity)

In [37]:
topN = 20
idx = user_similarity.argsort()[::-1][1:topN+1]
selected_user_similarity = user_similarity[idx]
selected_rating = rating_matrix_ar[idx]

avg_score = []
for col_idx in range(selected_rating.shape[1]):
    weight_score = sum(selected_rating[:, col_idx] * selected_user_similarity)
    similarity_sum = \
     sum(selected_user_similarity[selected_user_similarity > 0])
    avg_score.append(weight_score / similarity_sum)
avg_score = np.array(avg_score)

recommend_num = 5
counter = 0
for recommended_topic in topic_list[avg_score.argsort()[::-1]]:
    if recommended_topic not in already_followed_topic:
        print(recommended_topic)
        counter += 1
        if recommend_num <= counter:
            break

インテリジェント・デザイン
仕事術
マーケティング
でんぱ組.inc
セキュリティホール


In [43]:
topN = 20
idx = user_similarity.argsort()[::-1][1:topN+1]
selected_user_similarity = user_similarity[idx]
selected_rating = rating_matrix_ar[idx]

avg_score = []
for col_idx in range(selected_rating.shape[1]):
    weight_score = sum(selected_rating[:, col_idx] * selected_user_similarity)
    similarity_sum = \
     sum(selected_user_similarity[selected_user_similarity > 0])
    avg_score.append(weight_score / similarity_sum)
avg_score = np.array(avg_score)

recommend_num = 5
counter = 0
for recommended_topic in topic_list[avg_score.argsort()[::-1]]:
    if recommended_topic not in already_followed_topic:
        print(recommended_topic)
        counter += 1
        if recommend_num <= counter:
            break

転職
でんぱ組.inc
Twitter
インテリジェント・デザイン
PlayStation_4
